In [ ]:
#hide
#default_exp tools.metric_visualization
from nbdev.showdoc import show_doc

# Metric visualization

> Visualize metrics collected from experiments.

In [ ]:
#export
import warnings
warnings.filterwarnings('ignore')

import argparse
import sys
sys.path.append('.')
sys.path.append('src')
import pandas as pd
import pickle
import hpsearch.visualization.experiment_visualization as ev
from hpsearch.config.hpconfig import get_path_experiments
from hpsearch.config.hpconfig import get_default_operations

def metric_visualization (exps=[-1,-2], run_number=0, root_folder=None, metric=None, op = None,
                         parameters=None, name_file='model_history.pk', include_loss=False, backend='visdom'):
    
    default_operations = get_default_operations ()
    if root_folder is None:
        root_folder = default_operations.get('root', 'results')
    if metric is None:
        metric = default_operations.get('metric', 'accuracy')
    if op is None:
        op = default_operations.get('op', 'min')
        
    # metrics
    if include_loss:
        metrics = [metric, 'loss']
    else:
        metrics = [metric]
        
    for i in range(len(exps)):
        if exps[i] == -1:
            root_path = get_path_experiments (folder = root_folder)
            experiment_number = pickle.load(open('%s/current_experiment_number.pkl' %root_path,'rb'))
            exps[i] = experiment_number

        if exps[i] == -2:
            root_path = get_path_experiments (folder = root_folder)
            df = pd.read_csv('%s/experiments_data.csv' %root_path,index_col=0)
            if op=='max':
                exps[i] = df['0_%s' %metric].idxmax()
            else:
                exps[i] = df['0_%s' %metric].idxmin()
    
    ev.plot_multiple_histories(exps, run_number=run_number, root_folder=root_folder,metrics=metrics, 
                                   parameters=parameters, name_file=name_file, backend=backend)

def main():

    parser = argparse.ArgumentParser(description='show metrics about experiments') 
    # Datasets
    parser.add_argument('-e', nargs='+', default=[-1, -2], type=int,
                        help="experiments")
    parser.add_argument('--metric', nargs='+', type=str, default=None, help="metrics")
    parser.add_argument('--root', type=str, default=None)
    parser.add_argument('-l','--labels',nargs='+', default=None, type=str)
    parser.add_argument('--run', default=0, type=int)
    parser.add_argument('--op', default=None, type=str)
    parser.add_argument('--backend', default='visdom', type=str)
    parser.add_argument('--file', default='model_history.pk', type=str)

    pars = parser.parse_args()

    ev.plot_multiple_histories(pars.e, run_number=pars.run, root_folder=pars.root, metrics=pars.metric, 
                               parameters=pars.labels, name_file=pars.file, backend=pars.backend)
    



## Test

### Produce data

In [ ]:
#collapse_output
from hpsearch.examples.dummy_experiment_manager import remove_previous_experiments, run_multiple_experiments

remove_previous_experiments()
run_multiple_experiments(nruns=5, noise=0.1)

experiment script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line: 134
processing hyper-parameter 0 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
returning registered experiment manager
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7f6ef881c190>
running experiment 0
run number: 0

parameters:
	epochs=5,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.00031828880310058594
0 - validation_accuracy: 0.23020811911974692
0 - test_accuracy: 0.3304360394333248
finished experiment 0
processing hyper-parameter 0 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 0
run number: 1

parameters:
	epochs=5,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.0002727

current path: /mnt/athena/hpsearch/nbs/tools
current path: /mnt/athena/hpsearch/nbs/tools
current path: /mnt/athena/hpsearch/nbs/tools
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1


running experiment 0
run number: 4

parameters:
	epochs=5,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.0002944469451904297
4 - validation_accuracy: 0.2396865460298531
4 - test_accuracy: 0.4667883226437229
finished experiment 0
processing hyper-parameter 1 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 0

parameters:
	epochs=5,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.0004074573516845703
0 - validation_accuracy: 0.6842022366162808
0 - test_accuracy: 0.4006583101511916
finished experiment 1
processing hyper-parameter 1 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 1

parameters:
	epochs=5,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experim

current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools


script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.00027632713317871094
3 - validation_accuracy: 0.2527123520499598
3 - test_accuracy: 0.32338329247825215
finished experiment 1
processing hyper-parameter 1 out of 9
doing run 4 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 4

parameters:
	epochs=5,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.00029087066650390625
4 - validation_accuracy: 0.6509084892611131
4 - test_accuracy: 0.4847985418307314
finished experiment 1
processing hyper-parameter 2 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 2
run number: 0

parameters:
	epochs=5,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.000281095504760

...

fitting model with 30 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
epoch 5: accuracy: 0.4800000000000001
epoch 6: accuracy: 0.5100000000000001
epoch 7: accuracy: 0.5400000000000001
epoch 8: accuracy: 0.5700000000000002
epoch 9: accuracy: 0.6000000000000002
epoch 10: accuracy: 0.6300000000000002
epoch 11: accuracy: 0.6600000000000003
epoch 12: accuracy: 0.6900000000000003
epoch 13: accuracy: 0.7200000000000003
epoch 14: accuracy: 0.7500000000000003
epoch 15: accuracy: 0.7800000000000004
epoch 16: accuracy: 0.8100000000000004
epoch 17: accuracy: 0.8400000000000004
epoch 18: accuracy: 0.8700000000000004
epoch 19: accuracy: 0.9000000000000005
epoch 20: accuracy: 0.8700000000000004
epoch 21: accuracy: 0.8400000000000004
epoch 22: accuracy: 0.8100000000000004
epoch 23: accuracy: 0.7800000000000004
epoch 24: accuracy: 0.7500000000000003
epoch 25: accuracy: 0.720000000

running experiment 8
run number: 0

parameters:
	epochs=30,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.0008511543273925781
0 - validation_accuracy: 0.9946158339942315
0 - test_accuracy: 0.7365790709195311
finished experiment 8
processing hyper-parameter 8 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 1

parameters:
	epochs=30,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.0009796619415283203
1 - validation_accuracy: 0.8700644483351956
1 - test_accuracy: 0.7376461446555858
finished experiment 8
processing hyper-parameter 8 out of 9
doing run 2 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 2

parameters:
	epochs=30,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_expe

fitting model with 30 epochs
epoch 0: accuracy: 0.63
epoch 1: accuracy: 0.66
epoch 2: accuracy: 0.6900000000000001
epoch 3: accuracy: 0.7200000000000001
epoch 4: accuracy: 0.7500000000000001
epoch 5: accuracy: 0.7800000000000001
epoch 6: accuracy: 0.8100000000000002
epoch 7: accuracy: 0.8400000000000002
epoch 8: accuracy: 0.8700000000000002
epoch 9: accuracy: 0.9000000000000002
epoch 10: accuracy: 0.9300000000000003
epoch 11: accuracy: 0.9600000000000003
epoch 12: accuracy: 0.9900000000000003
epoch 13: accuracy: 1.0200000000000002
epoch 14: accuracy: 1.0500000000000003
epoch 15: accuracy: 1.0800000000000003
epoch 16: accuracy: 1.1100000000000003
epoch 17: accuracy: 1.1400000000000003
epoch 18: accuracy: 1.1700000000000004
epoch 19: accuracy: 1.2000000000000004
epoch 20: accuracy: 1.1700000000000004
epoch 21: accuracy: 1.1400000000000003
epoch 22: accuracy: 1.1100000000000003
epoch 23: accuracy: 1.0800000000000003
epoch 24: accuracy: 1.0500000000000003
epoch 25: accuracy: 1.020000000000

running experiment 8
run number: 4

parameters:
	epochs=30,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 134
time spent on this experiment: 0.00090789794921875
4 - validation_accuracy: 0.9269255110674015
4 - test_accuracy: 0.7252604156996494
finished experiment 8


fitting model with 30 epochs
epoch 0: accuracy: 0.63
epoch 1: accuracy: 0.66
epoch 2: accuracy: 0.6900000000000001
epoch 3: accuracy: 0.7200000000000001
epoch 4: accuracy: 0.7500000000000001
epoch 5: accuracy: 0.7800000000000001
epoch 6: accuracy: 0.8100000000000002
epoch 7: accuracy: 0.8400000000000002
epoch 8: accuracy: 0.8700000000000002
epoch 9: accuracy: 0.9000000000000002
epoch 10: accuracy: 0.9300000000000003
epoch 11: accuracy: 0.9600000000000003
epoch 12: accuracy: 0.9900000000000003
epoch 13: accuracy: 1.0200000000000002
epoch 14: accuracy: 1.0500000000000003
epoch 15: accuracy: 1.0800000000000003
epoch 16: accuracy: 1.1100000000000003
epoch 17: accuracy: 1.1400000000000003
epoch 18: accuracy: 1.1700000000000004
epoch 19: accuracy: 1.2000000000000004
epoch 20: accuracy: 1.1700000000000004
epoch 21: accuracy: 1.1400000000000003
epoch 22: accuracy: 1.1100000000000003
epoch 23: accuracy: 1.0800000000000003
epoch 24: accuracy: 1.0500000000000003
epoch 25: accuracy: 1.020000000000

In [ ]:
#hide
from nbdev.export import *
notebook2script(recursive=True)